In [3]:
# rag pipeline as suggested by llama 3.2
import pandas as pd
from transformers import AutoModelForSequenceClassification, AutoTokenizer

# Load the CSV file into a Pandas dataframe
df = pd.read_csv('articles.csv')

# Load a pre-trained Hugging Face model and tokenizer
model_name = "distilbert-base-uncased"
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Preprocess and tokenize the text data
max_length = 50  # Define maximum token length
tokenized_data = tokenizer(
    list(df['text']),  # Pass the text column
    padding=True,
    truncation=True,
    max_length=max_length,
    return_tensors="pt"
)

# Assuming 'label' column exists in your CSV
labels = list(df['label'])

# Use the model to perform text classification on your dataset
results = []
for idx, label in enumerate(labels):
    inputs = {
        "input_ids": tokenized_data['input_ids'][idx].unsqueeze(0),
        "attention_mask": tokenized_data['attention_mask'][idx].unsqueeze(0),
        # If the model is fine-tuned for classification, pass labels for training
    }
    
    # Forward pass through the model
    outputs = model(**inputs)
    logits = outputs.logits.detach().numpy()  # Detach logits from the computation graph
    
    # Store results
    results.append({'label': label, 'logit': logits.tolist()})

# Save the results to a CSV file or perform further analysis
pd.DataFrame(results).to_csv('results.csv', index=False)


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


KeyError: 'text'